In [1]:
import os
import pymongo
import openai
from llama_index import Document, VectorStoreIndex, StorageContext
from llama_index.storage.docstore import MongoDocumentStore
from llama_index.storage.index_store import MongoIndexStore
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.readers import SimpleMongoReader

print("True")

Done


In [2]:
#Create a new client and connect to the server
client = pymongo.MongoClient(os.getenv("MONGODB_URI"))
documents = [Document(text='Sports'), Document(text='Grocery'), Document(text='Clothing')]
print("True")

Done


In [3]:
# create Atlas as a vector store
store = MongoDBAtlasVectorSearch(client, db_name=os.getenv('MONGODB_DB2'),collection_name=os.getenv('MONGODB_COLL2'),index_name=os.getenv('MONGODB_VECTOR_INDEX'))
storage_context = StorageContext.from_defaults(vector_store=store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)
response = index.as_query_engine().query('Suggest a department for a basketball')
print(response)


Generating embeddings: 100%|██████████| 3/3 [00:00<00:00,  3.91it/s]


Sports Equipment Department
